https://cloud.yandex.ru/docs/storage/tools/boto

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-uploading-files.html

In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import os
import sys
import threading

In [2]:
session = boto3.session.Session()

In [3]:
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net'
)

In [4]:
response = s3.list_buckets()
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  ecomru.tmpfs


In [5]:
# s3.put_object(Bucket='ecomru.tmpfs', Key='YCAJEcf7xCvFmawEmP7NQxPN0', Body='TEST', StorageClass='COLD')

In [6]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    try:
        response = s3.upload_file(file_name, bucket, object_name, Callback=ProgressPercentage(file_name))
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [7]:
class ProgressPercentage(object):

    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self._seen_so_far = 0
        self._lock = threading.Lock()

    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            sys.stdout.write(
                "\r%s  %s / %s  (%.2f%%)" % (
                    self._filename, self._seen_so_far, self._size,
                    percentage))
            sys.stdout.flush()

In [8]:
path = ''
files = ['ozon_performance.py', 'Parser.py']

In [9]:
for f in files:
#     print(path+f)
    file = path+f
    upload_file(file, 'ecomru.tmpfs')    

Parser.py  4319 / 4319.0  (100.00%)0  (100.00%)

In [10]:
# Получить список объектов в бакете
for key in s3.list_objects(Bucket='ecomru.tmpfs')['Contents']:
    print(key['Key'])

Parser.py
ozon_performance.py
